In [1]:
# -"- coding: utf-8 -"-
from __future__ import division
import re
from gensim import corpora, models, similarities
import pickle
import numpy as np
import pandas as pd
import os
import operator
os.getcwd()

'C:\\Users\\Tariq\\Dropbox (MIT)\\Projects\\3. ITS\\Tweet Topics-Demographics Corr'

In [16]:
f = open('all.txt','rb') 
gTweets = f.readlines()
gText, lat, lon, gTextraw = [], [], [], []
for tweet in gTweets:
    gText.append(tweet.decode('utf-8').split('"text": "')[1].split('", "place":')[0])
    lat.append(str(tweet).split('"coordinates": [ ')[1].split(',')[0])
    lon.append(str(tweet).split('"coordinates": [ ')[1].split(',')[1].split(' ]')[0][1:])
    gTextraw.append(tweet.decode('utf-8').split('"text": "')[1].split('", "place":')[0])

In [ ]:
for i in range(0,len(gText)):
    gText[i] = gText[i].replace("ـ","")
    gText[i] = re.findall("[ء-غف-ي]+",gText[i])

n_words = []
for text in gText:
    n_words.append(len(text))

In [ ]:
f2 = open('tweets3_tokens.csv','wb')
f2.write(('lat,lon,n_words,words\n').encode('utf-8'))
for i in range(0,len(gText)):
    if lat[i][0] == '[':
        continue
    f2.write((str(lat[i])+', '+str(lon[i])+', '+str(n_words[i])+',').encode('utf-8'))
    for text in gText[i]:
        f2.write((' '+text).encode('utf-8'))
    f2.write('\n'.encode('utf-8'))
f2.close()

In [18]:
stopwords = open("C:\\Users\\Tariq\\Dropbox (MIT)\\Projects\\3. ITS\\Arabic NLP Resources\\stopwords.txt","rb").readlines()
for i in range(len(stopwords)):
    stopwords[i] = stopwords[i].decode('utf8')
    stopwords[i] = stopwords[i].split("\r")[0]
stopwords = stopwords[1:]

In [2]:
twt = pd.read_csv('tweets3_tokens_nei.csv',encoding='utf-8')
words = twt['words'].values
nei = twt['NEIGHBORH1'].values
n_words= twt['n_words'].values

In [45]:
words[0].replace('آ','ا').replace('أ','ا').split()
texts[0]

corpus_lda[0]

[(20, 0.16833333333333367),
 (46, 0.50166666666666759),
 (56, 0.16833333333333367)]

In [19]:
# More Cleaning: Removing 'NaN', Normalizing some characters, Removing stop words
cln_words = []
for i in range(0,len(words)):
    if words[i] != words[i]:     #Removing NaNs in words
        words[i] = ''
    if nei[i] != nei[i]:         #Removing NaNs in nei
        nei[i] = ''
    cln_word = []
    for word in words[i].split():
        if word not in stopwords and len(word) > 2:
            word = word.replace("أ","ا")
            word = word.replace("إ","ا")
            word = word.replace("اً","ا")
            word = word.replace("ؤ","و")
            word = word.replace("ئ","ي")
            word = word.replace("ة","ه")
            cln_word.append(word)
    cln_words.append(cln_word)

In [ ]:
#lens = [len(tweet) for tweet in cln_words]
counts = [1 for tweet in lens if tweet > 9]
sum(counts), len(lens)

In [ ]:
#word_clusters2 = defaultdict(list)

word_clusters = {}
for n in nei:
    word_clusters[n] = []
del word_clusters['']

for i in range(0,len(nei)):
    for word in cln_words[i]:
        if nei[i] != '':
            word_clusters[nei[i]].append(word)
            #word_clusters2[nei[i]].append(word)

In [ ]:
c = 0
for n in word_clusters.keys():
    if len(set(word_clusters[n])) > 6000:
        c+=1
for key in word_clusters.keys():
    if len(word_clusters[key]) == 0:
        print(word_clusters[key])
word_clusters['AL-AMAJIYAH'],word_clusters['AL-WESITA']

In [ ]:
# Print Words Distribution for a given neighborhood to have an idea of how much a single word is repeated!
len(word_clusters['AR-RAWABI']),len(set(word_clusters['AR-RAWABI']))
frequency = defaultdict(int)
for word in word_clusters['AR-RAWABI']:
    frequency[word] += 1
s = sorted(frequency.items(), key=operator.itemgetter(1), reverse=True)
s

In [23]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in cln_words:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in cln_words]

In [ ]:
sum([1 for tweet in texts if len(tweet)>10]),sum([1 for tweet in texts if len(tweet)==0]),len(texts)

In [25]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=100)
corpus_lda = lda[corpus]

topics = lda.show_topics(100,100)
#lda.log_perplexity(corpus)       =  -10.337044669545444
# topic = lda.show_topic(95,500)

In [ ]:
# Extract Features from Geotagged Tweets    (i.e. apply LDA model)
neighborhoods = word_clusters.keys()
word_clusters_corpus = [dictionary.doc2bow(word_clusters[text]) for text in neighborhoods]
word_clusters_lda = lda[word_clusters_corpus]

In [ ]:
# topic distributions per documents with unused topics set to zero
nei_topics = [[]] * len(word_clusters_lda)
j = 0
for doc in word_clusters_lda:
    nei_topics[j] = [0] * lda.num_topics
    i = 0
    while(i<len(doc)):
        temp = doc[i]
        nei_topics[j][temp[0]]=format(temp[1],'.4f')
        i = i + 1
    j = j + 1

In [ ]:
# export to perform correlation in R
nei_ = pd.DataFrame(nei_topics)
nei_['neighborhood'] = neighborhoods
nei_.to_csv('nei_topic.csv')

In [ ]:
# exporting top 100 words for each topic to generate wordclouds of selected topics
topic_words = pd.DataFrame()
for i in range(0,100):
    word, freq = [], []
    for row in lda.show_topic(i,100):
        word.append(row[0])
        freq.append(row[1])
    freq_mult = 1/float(freq[-1])
    freq = [float(value) * freq_mult for value in freq]
    topic_words['word_'+str(i)] = word
    topic_words['freq_'+str(i)] = freq

topic_words.to_excel('topic_words.xlsx', encoding='utf-8')

In [ ]:
nei_topics

In [ ]:
neighborhoods

In [ ]:
# Simple WordCloud
from os import path
from scipy.misc import imread
import matplotlib.pyplot as plt
import random

from wordcloud import WordCloud, STOPWORDS

text = ''
wordcloud = WordCloud(relative_scaling = 1.0, stopwords = 'to of').generate(text)

plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
#Dump





# another way to read a unicode file
f = codecs.open("tweets.txt", "r", "utf-8")
lines = f.readlines()
re.sub("[^ء-غف-ي ]", "", lines[224470])

#
export = pd.DataFrame({'lat':lat,'lon':lon,'n_words':n_words})
export.to_csv('word_counts.csv')

In [ ]:
len(cln_words),len(n_words),len(nei)

word_clusters = {}
for n in nei:
    word_clusters[n] = []
del word_clusters['']

for i in range(0,len(nei)):
    for word in cln_words[i]:
        if nei[i] != '':
            word_clusters[nei[i]].append(word)

In [ ]:
word_counts = {}
for n in nei:
    word_counts[n] = 0
#del word_counts['']

for i in range(0,len(nei)):
    if nei[i] != '':
        word_counts[nei[i]] += n_words[i]

In [ ]:
tweet_counts = {}
for n in nei:
    tweet_counts[n] = 0
#del word_counts['']

for i in range(0,len(nei)):
    if nei[i] != '':
        tweet_counts[nei[i]] += 1

In [ ]:
word_counts